In [1]:
from gym import Env
import random
from gym.spaces import Discrete, Box
import numpy as np

In [2]:
class ShowerEnv(Env):
    
    def __init__(self):
#         actions we can take i.e. up, down, stay
        self.action_space = Discrete(3)
#       Temperature Array
        self.observation_space = Box(low=np.array([0]), 
                                                  high=np.array([100]))
#       Set Start Temperature
        self.state = 38 + random.randint(-3,3)
#       Set shower length
        self.shower_length = 60
    
    def step(self, action):
        # Apply actions
        self.state += action-1
        # Reduce shower length by 1 second
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >=37 and self.state<=39:
            reward = 1
        else:
            reward = -1
        
        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        # Apply temperature noise
        self.state += random.randint(-1,1)
        # set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
            
    def render(self):
        # Implement Visualization
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60
        
        return self.state

In [3]:
env = ShowerEnv()

c:\users\vishwa\appdata\local\programs\python\python37\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env.action_space.sample()

2

In [5]:
env.observation_space.sample()

array([56.181973], dtype=float32)

In [6]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
     
    while not done:
#         env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print('Episode:{}, Score:{}'.format(episode, score))
# env.close()

Episode:1, Score:-50
Episode:2, Score:-60
Episode:3, Score:-52
Episode:4, Score:-52
Episode:5, Score:-40
Episode:6, Score:-52
Episode:7, Score:-14
Episode:8, Score:-24
Episode:9, Score:-56
Episode:10, Score:-36


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(states, action):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    
    return model

In [9]:
states = env.observation_space.shape
actions = env.action_space.n
print(state, actions)

36 3


In [14]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [15]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [16]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [17]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5e4, visualize=False, verbose=1)

Training for 50000.0 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 24:24 - reward: 1.0000

c:\users\vishwa\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
c:\users\vishwa\appdata\local\programs\python\python37\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 55s 5ms/step - reward: -0.6066
166 episodes - episode_reward: -36.337 [-60.000, 22.000] - loss: 1.095 - mae: 6.296 - mean_q: -7.588

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: -0.6014
167 episodes - episode_reward: -36.132 [-60.000, 28.000] - loss: 2.064 - mae: 10.486 - mean_q: -15.045

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: -0.6520
167 episodes - episode_reward: -39.126 [-60.000, 8.000] - loss: 2.714 - mae: 12.391 - mean_q: -17.798

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: -0.5780
166 episodes - episode_reward: -34.747 [-60.000, 20.000] - loss: 2.302 - mae: 11.391 - mean_q: -16.375

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: -0.4408
done, took 286.561 seconds


In [18]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 56.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 54.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 58.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 56.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 58.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 56.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 58.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 54.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st